In [1]:
# 1) CARGAR EL DATAFRAME COMBINADO Y DETECTAR COLUMNAS CON MEZCLA DE TIPOS

import pandas as pd

# Cargar con low_memory=False para evitar advertencias durante la inferencia de tipos
df = pd.read_csv('data/combined.csv', low_memory=False)

# Verificar el dataframe cargado:
filas_cargadas = df.shape[0]  # obtener el número de filas cargadas
print(f"DataFrame cargado: {filas_cargadas} filas.")
print(f"Columnas en el DataFrame: {df.shape[1]}")

# Detectar columnas con mezcla de tipos usando 'object' que deberían ser numéricas
mixed_type_cols = []

for col in df.columns:
    # Si tiene más de un tipo de dato (ej. int y str)
    tipos = df[col].map(type).value_counts()
    if len(tipos) > 1:
        mixed_type_cols.append((col, tipos))

# Mostrar un resumen
if mixed_type_cols:
    print("Columnas con tipos de datos mezclados detectadas:")
    for col, tipos in mixed_type_cols:
        print(f"Columna: '{col}' - Tipos detectados: {tipos.to_dict()}")
else:
    print("No se detectaron columnas con tipos de datos mezclados.")

DataFrame cargado: 38932 filas.
Columnas en el DataFrame: 223
No se detectaron columnas con tipos de datos mezclados.


In [2]:
# FASE 3. NORMALIZACION O ESTANDARIZACION (PREPARACION PARA EL MACHINE LEARNING)

#  0) VARIABLES CATEGORICAS (CADENAS DE TEXTO)

# GENERO: En la realidad de ITSON las mujeres (F) desertan menos que los hombres (M), por lo que se esperaría que tienen menor riesgo de deserción.
df['GENERO'] = df['GENERO'].astype(str).str.upper().map({'F': 0, 'M': 1})

# CARRERA: Se cree que un alumno condicionado tiene mayor riesgo de deserción.

# Verificación resultado
print(f"Columnas en el DataFrame: {df.shape[1]}")
print(df[['GENERO', 'CARRERA', 'RISK_CARRERA']].head())

Columnas en el DataFrame: 223
   GENERO CARRERA  RISK_CARRERA
0     1.0   LDCFD             0
1     0.0   LDCFD             0
2     0.0     MVZ             0
3     0.0   LCOPU             0
4     1.0     IEM             0


In [3]:
import numpy as np

# 1. Convertir una columna de tipo object a tipo string e imputar No aplica y Desconocido
# Lista de columnas a convertir a tipo string
columns_to_convert = [
    'EMPLID', 'CARRERA', 'UNIDAD', 'Ciudad', 
    'Estado', 'P052_O010V', 'P047_O008V', 'P006_O008V',
    'P006_O009V', 'P010_O002V', 'P060_O002V', 'P005_O002V'
]

# Imputación de valores para columnas específicas (ya imputadas en el notebook anterior)
# Columnas a rellenar con "No aplica" por ser las opciones de "Especifica" (preguntas abiertas)
columns_no_aplica = [
    'P006_O008V', 'P006_O009V', 'P052_O010V', 'P047_O008V',
    'P060_O002V', 'P010_O002V', 'P005_O002V'
]

# Columnas a rellenar con "Desconocido" 
columns_desconocido = [
    'EMPLID', 'CARRERA', 'UNIDAD', 'GENERO', 'Ciudad', 'Estado'
]

# Rellenar valores nulos con "No aplica"
df[columns_no_aplica] = df[columns_no_aplica].fillna("No aplica")

# Rellenar valores nulos con "Desconocido"
df[columns_desconocido] = df[columns_desconocido].fillna("Desconocido")

# Convertir todas las columnas a tipo string después de la imputación
df[columns_to_convert] = df[columns_to_convert].astype('string')


# 2.Imputar con 0 las respuestas a las preguntas: P008_O002V (Cuantos cigarrillos fumas); P013_O001V (Cuantas horas trabajas); P018_O002V (Cuantos hijos tienes)

cols_to_impute = ['P008_O002V', 'P013_O001V', 'P018_O002V']
for col in cols_to_impute:
    # Verificar valores perdidos antes de la imputación
    missing_before = df[col].isna().sum()
    
    # Mostrar valores únicos no nulos antes de imputar

    # Reemplazar valores no numéricos o vacíos por NaN
    df[col] = df[col].replace(['---', 'N/A', 'n/a', 's/n', '', ' '], np.nan)

    # Imputar valores NaN con 0
    df[col] = df[col].fillna(0)

    # Convertir a tipo numérico
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Rellenar Valores Nulos para el resto de las variables

# Identificar columnas con valores nulos
columnas_con_nulos = df.columns[df.isnull().any()]

# Contadores por tipo de imputación
cols_imputadas_cero = []
cols_imputadas_NA = []
cols_imputadas_desconocido = []
cols_imputadas_moda = []

# Guardar la columna ETNIA antes de la imputación
df_etnia = df[['ETNIA']].copy()

# Rellenar valores nulos según grupo
for col in columnas_con_nulos:  
    if col in ['P014_O001V', 'P015_O002V', 'P015_O001V', 'P015_O003V', 'P015_O004V',
               'P015_O005V', 'P016_O001V', 'P043_O001V', 'P042_O001V', 'P027_O001V', 'P027_O002V',
               'P027_O003V', 'P027_O004V', 'P027_O005V', 'P027_O006V', 'P027_O007V',
               'P027_O008V', 'P027_O009V', 'P027_O010V', 'P027_O011V', 'P027_O012V',
               'P027_O013V', 'P027_O014V', 'P026_O001V', 'P024_O001V', 'P024_O002V',
               'P018_O001V', 'P044_O001V', 'P008_O001V', 'P009_O001V', 'P007_O001V',
               'P010_O001V', 'P006_O001V', 'P006_O002V', 'P006_O003V', 'P006_O004V', 
               'P006_O005V', 'P006_O006V', 'P006_O007V', 'P019_O001V']:
        df[col] = df[col].fillna("0")
        cols_imputadas_cero.append(col)

    elif col in ['P051_O001V', 'P051_O002V', 'P051_O003V', 'P051_O004V', 'P051_O005V', 
                 'P011_O001V', 'P017_O001V', 'P039_O001V', 'P048_O001V', 'P048_O002V', 
                 'P048_O003V', 'P048_O004V', 'P048_O005V', 'P048_O006V']:
        df[col] = df[col].fillna("NA")
        cols_imputadas_NA.append(col)

    elif col in ['P057_O001V', 'P058_O001V', 'P059_O001V', 'P059_O002V', 'P060_O001V', 
                 'P020_O001V', 'P022_O001V']:
        df[col] = df[col].fillna("Desconocido")
        cols_imputadas_desconocido.append(col)

    else:
        mode_value = df[col][df[col] != 0].mode(dropna=True).iloc[0] if not df[col][df[col] != 0].mode(dropna=True).empty else None
        if mode_value is not None:
            df[col] = df[col].fillna(mode_value)
            cols_imputadas_moda.append(col)

# Restaurar ETNIA
df['ETNIA'] = df_etnia['ETNIA']

# Verificación
nulos_restantes = df.isna().sum()
nulos_finales = nulos_restantes[nulos_restantes > 0]

if nulos_finales.empty:
    print("No hay columnas con valores nulos después de la imputación.")
else:
    print("Aún hay columnas con valores nulos:")
    print(nulos_finales)

Aún hay columnas con valores nulos:
ETNIA    24835
dtype: int64


In [4]:
# 1) VARIABLES CUALITATIVAS NOMINALES A CODIFICAR CON ONE HOT ENCODING

# Lista  de variables categóricas nominales (con varias respuestas)
variables_categoricas = list(set([
    'P009_O001V', 'P010_O001V', 'P014_O001V', 'P017_O001V', 'P020_O001V', 'P022_O001V', 'P025_O001V', 'P031_O001V', 'P031_O002V', 'P031_O003V', 
    'P032_O001V', 'P032_O002V', 'P032_O003V', 'P053_O001V', 
    'P054_O001V', 'P058_O001V', 'P060_O001V', 
]))

# Verificar valores nulos solo en las variables categóricas seleccionadas
valores_nulos_categoricas = df[variables_categoricas].isnull().sum()
columnas_con_nulos_categoricas = valores_nulos_categoricas[valores_nulos_categoricas > 0]

# Mensaje claro según el resultado
if columnas_con_nulos_categoricas.empty:
    print("Las variables categóricas seleccionadas para One Hot Encoding no tienen valores nulos.")
else:
    print("Las siguientes variables categóricas tienen valores nulos y podrían causar errores en One Hot Encoding:")
    print(columnas_con_nulos_categoricas)

# Conteo de otros valores nulos en el dataframe
valores_nulos = df.isnull().sum()
columnas_con_nulos = valores_nulos[valores_nulos > 0]
if columnas_con_nulos.empty:
    print("No hay columnas con valores nulos en otras columnas del DataFrame")
else:
    print("Número de valores nulos en otras columnas del DataFrame:")
    print(columnas_con_nulos)

Las variables categóricas seleccionadas para One Hot Encoding no tienen valores nulos.
Número de valores nulos en otras columnas del DataFrame:
ETNIA    24835
dtype: int64


In [5]:
# Contar número de columnas antes del One-Hot Encoding
num_columnas_antes = df.shape[1]
print(f"Número de columnas antes del One-Hot Encoding: {num_columnas_antes}")

# Guardar nombres de columnas antes del One-Hot Encoding
columnas_antes = set(df.columns)

# Codificar las variables categóricas (One-Hot Encoding)
variables_categoricas_presentes = [col for col in variables_categoricas if col in df.columns]

for columna in variables_categoricas_presentes:
    dummies = pd.get_dummies(df[columna], prefix=columna, drop_first=False)
    df = pd.concat([df, dummies], axis=1)
    df.drop(columna, axis=1, inplace=True)

# Guardar nombres de columnas después del One-Hot Encoding
columnas_despues = set(df.columns)

# Determinar columnas nuevas creadas por el One-Hot Encoding
columnas_creadas = columnas_despues - columnas_antes

print(f"Columnas nuevas creadas ({len(columnas_creadas)}):")
print(sorted(columnas_creadas))

# Contar número de columnas después del One-Hot Encoding
num_columnas_despues = df.shape[1]
nuevas_columnas = num_columnas_despues - num_columnas_antes
print(f"Número de columnas nuevas agregadas (no repetidas): {nuevas_columnas}")
print(f"Número total de columnas después del One-Hot Encoding: {num_columnas_despues}")

# Verificar valores nulos después de la codificación
valores_nulos_actualizados = df.isnull().sum()
valores_nulos_presentes = valores_nulos_actualizados[valores_nulos_actualizados > 0]

print("\nNúmero de valores nulos en el DataFrame después de la codificación:")
print(valores_nulos_presentes if not valores_nulos_presentes.empty else "No hay valores nulos.")

Número de columnas antes del One-Hot Encoding: 223
Columnas nuevas creadas (80):
['P009_O001V_0.0', 'P009_O001V_1.0', 'P009_O001V_2.0', 'P009_O001V_3.0', 'P009_O001V_4.0', 'P009_O001V_5.0', 'P009_O001V_6.0', 'P010_O001V_0.0', 'P010_O001V_1.0', 'P010_O001V_2.0', 'P010_O001V_3.0', 'P010_O001V_4.0', 'P010_O001V_5.0', 'P014_O001V_0.0', 'P014_O001V_1.0', 'P014_O001V_2.0', 'P014_O001V_3.0', 'P014_O001V_4.0', 'P014_O001V_5.0', 'P014_O001V_6.0', 'P017_O001V_1.0', 'P017_O001V_2.0', 'P017_O001V_3.0', 'P017_O001V_4.0', 'P017_O001V_5.0', 'P017_O001V_NA', 'P020_O001V_1.0', 'P020_O001V_2.0', 'P020_O001V_3.0', 'P020_O001V_4.0', 'P020_O001V_5.0', 'P020_O001V_6.0', 'P020_O001V_Desconocido', 'P022_O001V_1.0', 'P022_O001V_2.0', 'P022_O001V_3.0', 'P022_O001V_4.0', 'P022_O001V_5.0', 'P022_O001V_6.0', 'P022_O001V_7.0', 'P022_O001V_8.0', 'P022_O001V_Desconocido', 'P025_O001V_0.0', 'P025_O001V_1.0', 'P025_O001V_2.0', 'P025_O001V_3.0', 'P031_O001V_1.0', 'P031_O001V_2.0', 'P031_O001V_3.0', 'P031_O002V_1.0', 'P0

In [6]:
# Buscar columnas candidatas para conversión a entero
columnas_a_convertir = []

for col in df.columns:
    valores_unicos = df[col].dropna().unique()
    
    # Si todos los valores son booleanos, 0, 1 o '0', '1', se considera dummy
    if set(valores_unicos).issubset({0, 1, True, False, '0', '1'}):
        columnas_a_convertir.append(col)

# Convertir a entero
for col in columnas_a_convertir:
    # Reemplazar valores nulos con un valor predeterminado antes de la conversión
    df[col] = df[col].fillna(0).astype(int)

# Verificación
print(f"Se convirtieron {len(columnas_a_convertir)} columnas a enteros.")
print(df[columnas_a_convertir].dtypes)

Se convirtieron 183 columnas a enteros.
P005_O001V                int64
P006_O001V                int64
P006_O002V                int64
P006_O003V                int64
P006_O004V                int64
                          ...  
P020_O001V_3.0            int64
P020_O001V_4.0            int64
P020_O001V_5.0            int64
P020_O001V_6.0            int64
P020_O001V_Desconocido    int64
Length: 183, dtype: object


In [7]:
# 2) VARIABLES CUALITATIVAS ORDINALES

# Definir las columnas con las variables ordinales 
columns_to_check = list(set([
    'P011_O001V', 'P016_O001V', 'P019_O001V', 'P021_O001V', 'P023_O001V', 'P024_O001V', 
    'P024_O002V', 'P028_O001V', 'P029_O001V', 'P030_O001V', 'P033_O001V', 
    'P033_O002V', 'P033_O003V', 'P033_O004V', 'P033_O005V', 'P033_O006V', 'P033_O007V',
    'P033_O008V', 'P033_O009V', 'P033_O010V',
    'P034_O001V', 'P034_O002V', 'P034_O003V', 'P034_O004V', 'P034_O005V', 'P034_O006V', 
    'P034_O007V', 'P034_O008V', 'P034_O009V', 'P035_O001V', 'P035_O002V', 'P035_O003V', 
    'P035_O004V', 'P035_O005V', 'P036_O001V', 'P036_O002V', 'P036_O003V', 'P036_O004V', 
    'P036_O005V', 'P036_O006V', 'P036_O007V', 'P038_O001V', 'P040_O001V', 'P040_O002V', 
    'P045_O001V', 'P048_O001V', 'P048_O002V', 'P048_O003V', 'P048_O004V',
    'P048_O005V', 'P048_O006V', 'P049_O001V', 'P049_O002V', 'P049_O003V', 'P049_O004V', 
    'P049_O005V', 'P049_O006V', 'P049_O007V', 'P049_O008V', 'P049_O009V', 'P049_O010V', 
    'P049_O011V', 'P055_O001V', 'P055_O002V', 'P055_O003V', 'P055_O004V', 'P055_O005V',
    'P057_O001V', 'P059_O001V', 'P059_O002V', 'P039_O001V'
]))

# 1. Verificar valores nulos exclusivamente en esas columnas
nulos_ordinales = df[columns_to_check].isnull().sum()
columnas_con_nulos_ordinales = nulos_ordinales[nulos_ordinales > 0]

# Mensaje  según el resultado
if columnas_con_nulos_ordinales.empty:
    print("Las variables ordinales no tienen valores nulos.")
else:
    print("Las siguientes variables ordinales tienen valores nulos y podrían requerir imputación:")
    print(columnas_con_nulos_ordinales)

# Conteo de otros valores nulos en el dataframe
valores_nulos = df.isnull().sum()
columnas_con_nulos = valores_nulos[valores_nulos > 0]
if columnas_con_nulos.empty:
    print("No hay columnas con valores nulos en otras columnas del DataFrame")
else:
    print("Número de valores nulos en otras columnas del DataFrame:")
    print(columnas_con_nulos)

Las variables ordinales no tienen valores nulos.
No hay columnas con valores nulos en otras columnas del DataFrame


In [8]:
# 2. Reemplazar 'NA', 'Desconocido', o cualquier valor no válido por un valor válido (moda)
variables_sin_mapeo = list(set([
    'P011_O001V', # ¿Con qué frecuencia has perdido la noción del tiempo y después no recuerdas nada?
    'P039_O001V', # ¿Cómo valoras el desempeño de tus profesores del bachillerato?
    'P048_O002V', 'P048_O003V', 'P048_O004V', 'P048_O005V', 'P048_O006V', # ¿Con qué frecuencia utilizabas en el bachillerato los siguientes servicios?
    'P057_O001V', # ¿Cómo consideras tus posibilidades de encontrar trabajo relacionado con tu profesión?
    'P059_O001V', 'P059_O002V', # ¿Cómo percibes el desarrollo de tu vida profesional, una vez que concluyas tus estudios de licenciatura?
]))

for col in variables_sin_mapeo:
    if col in df.columns:
        if df[col].dtype == 'object':
            # Reemplaza 'NA' y 'Desconocido' con pd.NA
            df[col] = df[col].replace(['NA', 'Desconocido'], pd.NA)
            moda = df[col].mode(dropna=True)
            if not moda.empty:
                df[col] = df[col].fillna(moda[0])
            else:
                df[col] = df[col].fillna("Otro")
        else:
            # Asegura tipo numérico y reemplaza NaN con la moda
            df[col] = pd.to_numeric(df[col], errors='coerce')
            moda = df[col].mode(dropna=True)
            if not moda.empty:
                df[col] = df[col].fillna(moda[0])
            else:
                df[col] = df[col].fillna(0)

# Verificar los valores únicos después del reemplazo
print(df[variables_sin_mapeo].isnull().sum())
print("\nValores únicos en las columnas después de reemplazo:")
for column in variables_sin_mapeo:
    print(f"{column}: {df[column].unique()}")

P048_O004V    0
P057_O001V    0
P048_O003V    0
P048_O006V    0
P011_O001V    0
P039_O001V    0
P059_O002V    0
P059_O001V    0
P048_O002V    0
P048_O005V    0
dtype: int64

Valores únicos en las columnas después de reemplazo:
P048_O004V: [1. 2. 3. 0. 4.]
P057_O001V: ['1.0' '2.0' '3.0' '4.0']
P048_O003V: [1. 2. 3. 4. 0.]
P048_O006V: [1. 0. 3. 2. 4.]
P011_O001V: [1. 4. 2. 6. 3. 5.]
P039_O001V: [1. 2. 3. 4.]
P059_O002V: ['3.0' '2.0' '1.0' '4.0' '5.0']
P059_O001V: ['3.0' '2.0' '1.0' '4.0' '5.0']
P048_O002V: [1. 3. 4. 0. 2.]
P048_O005V: [1. 2. 3. 4. 0.]


In [9]:
# 3. Verificar el rango de los valores en las variables ordinales
for column in columns_to_check:
    df[column] = pd.to_numeric(df[column], errors='coerce')  # Convierte a numérico, y pone NaN en valores no convertibles
print("\nVerificación del rango de valores en las columnas ordinales después de la conversión:")
for column in columns_to_check:
    min_value = df[column].min()
    max_value = df[column].max()
    print(f"{column}: Min = {min_value}, Max = {max_value}")


Verificación del rango de valores en las columnas ordinales después de la conversión:
P049_O007V: Min = 1.0, Max = 4.0
P048_O004V: Min = 0.0, Max = 4.0
P034_O004V: Min = 1.0, Max = 4.0
P057_O001V: Min = 1.0, Max = 4.0
P059_O001V: Min = 1.0, Max = 5.0
P033_O001V: Min = 1, Max = 4
P048_O001V: Min = 0, Max = 4
P033_O003V: Min = 1.0, Max = 4.0
P049_O008V: Min = 1.0, Max = 4.0
P023_O001V: Min = 1.0, Max = 5.0
P024_O002V: Min = 0.0, Max = 12.0
P036_O001V: Min = 1.0, Max = 4.0
P036_O006V: Min = 1.0, Max = 4.0
P049_O002V: Min = 1.0, Max = 4.0
P055_O004V: Min = 0.0, Max = 4.0
P033_O006V: Min = 1.0, Max = 4.0
P040_O001V: Min = 1.0, Max = 4.0
P016_O001V: Min = 0.0, Max = 4.0
P049_O009V: Min = 1.0, Max = 4.0
P034_O006V: Min = 1.0, Max = 4.0
P028_O001V: Min = 1.0, Max = 3.0
P035_O004V: Min = 1.0, Max = 4.0
P049_O005V: Min = 1.0, Max = 4.0
P034_O003V: Min = 1.0, Max = 4.0
P034_O001V: Min = 1.0, Max = 4.0
P055_O005V: Min = 0.0, Max = 4.0
P030_O001V: Min = 1.0, Max = 3.0
P035_O005V: Min = 1.0, Max = 

In [10]:
# 4. Importar y crear un objeto MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Crear un objeto MinMaxScaler
scaler = MinMaxScaler()

# Aplicar el MinMaxScaler solo a las columnas ordinales
df[columns_to_check] = scaler.fit_transform(df[columns_to_check])

# Verificar los primeros registros para confirmar la normalización
print(df[columns_to_check].head())

# Verificar los resultados después de la codificación ordinal
print("\nVerificación de los valores después de Ordinal Encoding:")
for column in columns_to_check:
    print(f"{column}: {df[column].unique()}")

   P049_O007V  P048_O004V  P034_O004V  P057_O001V  P059_O001V  P033_O001V  \
0    1.000000        0.25    1.000000    0.000000        0.50         0.0   
1    0.666667        0.25    1.000000    0.000000        0.25         0.0   
2    0.000000        0.25    0.333333    0.333333        0.50         0.0   
3    1.000000        0.25    0.666667    0.333333        0.25         0.0   
4    0.666667        0.50    0.666667    0.000000        0.00         0.0   

   P048_O001V  P033_O003V  P049_O008V  P023_O001V  ...  P034_O005V  \
0        0.25    0.333333    0.666667        1.00  ...    0.333333   
1        0.25    0.000000    0.333333        0.25  ...    0.666667   
2        0.25    0.000000    0.000000        0.00  ...    0.666667   
3        0.25    0.333333    1.000000        0.50  ...    0.666667   
4        0.75    0.333333    1.000000        0.25  ...    0.666667   

   P048_O006V  P049_O001V  P055_O003V  P045_O001V  P033_O005V  P033_O007V  \
0        0.25    1.000000        0.25  

In [11]:
#  3) VARIABLES CUANTITATIVAS
### Normalización (rango 0 a 1) de variables cuantitativas

# Asegurar consistencia en P013_O001V: si NO trabaja, entonces horas trabajadas = 0
df.loc[df['P012_O001V'] == '0', 'P013_O001V'] = 0
df['P013_O001V'] = pd.to_numeric(df['P013_O001V'], errors='coerce').fillna(0)
df.drop(columns=['P012_O001V'], inplace=True)

# Desfragmentar el DataFrame antes de agregar nuevas columnas
df = df.copy()

# Copiar columnas originales a preservar
df['EDAD_ESCALADA'] = df['EDAD']
df['BACH_PROMEDIO_ESCALADO'] = df['BACH_PROMEDIO']

# Lista de variables cuantitativas a normalizar
variables_cuantitativas = ['P013_O001V', 'P044_O001V', 'P018_O002V', 'P008_O002V',
                           'Distancia_Campus', 'BACH_PROMEDIO_ESCALADO', 'PUNTAJE_ADM', 'EDAD_ESCALADA']

# Crear el objeto MinMaxScaler
scaler = MinMaxScaler()

# Aplicar la normalización
df[variables_cuantitativas] = scaler.fit_transform(df[variables_cuantitativas])

# Mostrar verificación
print("\nPrimeras filas con columnas normalizadas:")
print(df[variables_cuantitativas].head())


Primeras filas con columnas normalizadas:
   P013_O001V  P044_O001V  P018_O002V  P008_O002V  Distancia_Campus  \
0    0.190476    0.005722         0.0         0.0          0.010148   
1    0.000000    0.001431         0.2         0.0          0.010148   
2    0.000000    0.014306         0.0         0.0          0.704880   
3    0.285714    0.007153         0.0         0.0          0.010148   
4    0.714286    0.002861         0.0         0.0          0.010148   

   BACH_PROMEDIO_ESCALADO  PUNTAJE_ADM  EDAD_ESCALADA  
0                   0.819     0.397619       0.214286  
1                   0.000     0.471429       0.285714  
2                   0.740     0.504464       0.071429  
3                   0.859     0.580952       0.053571  
4                   0.809     0.466369       0.053571  


In [12]:
# Asegurar que no haya valores nulos en las variables 
valores_nulos = df.isnull().sum()
columnas_con_nulos = valores_nulos[valores_nulos > 0]
if columnas_con_nulos.empty:
    print("No hay columnas con valores nulos en el DataFrame")
else:
    print("Número de valores nulos en cada columna con valores nulos:")
    print(columnas_con_nulos)

No hay columnas con valores nulos en el DataFrame


In [13]:
# Primero convertir las columnas numéricas a tipo float
numeric_columns = ['P051_O001V', 'P051_O002V', 'P051_O003V', 'P051_O004V', 'P051_O005V']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Reemplazar valores nulos con la moda de cada columna
for col in numeric_columns:
    df[col].fillna(df[col].mode(), inplace=True)

# Reemplazar valores nulos con la moda de cada columna
df['GENERO'] = df['GENERO'].replace('nan', df['GENERO'].mode()[0])

# Verificar nuevamente los valores nulos para confirmar el reemplazo
valores_nulos = df.isnull().sum()
columnas_con_nulos = valores_nulos[valores_nulos > 0]

if columnas_con_nulos.empty:
    print("No hay columnas con valores nulos en el DataFrame")
else:
    print("Número de valores nulos en cada columna con valores nulos:")
    print(columnas_con_nulos)

No hay columnas con valores nulos en el DataFrame


In [14]:
# BORRAR COLUMNAS QUE NO SE NORMALIZARON

# Contar columnas antes de la eliminación
columnas_antes = df.shape[1]

# Lista de columnas a eliminar 
columnas_a_eliminar = [
    "P005_O002V", "P006_O008V", "P006_O009V", "P010_O002V", # Preguntas abiertas
    "P047_O008V", "P052_O010V", "P060_O002V",  # Preguntas abiertas
    "PUNTAJE_ADM",  # Porque cambió la escala cuando ya no se aplica examen de admisión
    "ADEUDO"        # Porque solo se tiene para los estudiantes que desertaron
]

# Eliminar las columnas del DataFrame
df.drop(columns=columnas_a_eliminar, inplace=True, errors='ignore')

# Contar columnas después de la eliminación
columnas_despues = df.shape[1]

# Imprimir resultado
print(f"Se eliminaron {columnas_antes - columnas_despues} columnas.")

Se eliminaron 9 columnas.


In [15]:
import os

# Guardar el dataframe combinado en un archivo nuevo
folder_path = './data'  
df.to_csv(os.path.join(folder_path, 'encoded.csv'), index=False)

# Verificar
filas_cargadas = df.shape[0] 
print(f"DataFrame guardado: {filas_cargadas} filas.")
print(f"Columnas en el DataFrame: {df.shape[1]}")

DataFrame guardado: 38932 filas.
Columnas en el DataFrame: 278


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=24fa97d8-6b94-4f62-a2dc-97416a953ae1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>